In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

Loading imports


In [2]:
torch.cuda.set_device(2)

In [3]:
dls = get_3d_dls_aug(Meta.df_comb, sz=128, bs=32, grps=Meta.grps_stg1)

In [4]:
x,y = dls.one_batch()
x.shape, y.shape

(torch.Size([32, 60, 3, 128, 128]), torch.Size([32, 60, 6]))

## Model

In [5]:
class SeqHead(nn.Module):
    def __init__(self):
        super().__init__()
        d_model = 2048*2
        self.conv = nn.Sequential(
            ConvLayer(512, 1024, stride=2),
            ConvLayer(1024, 1024, stride=2),
            ConvLayer(1024, d_model, stride=2),
            AdaptiveConcatPool2d(),
#             nn.AdaptiveAvgPool2d(1),
            Flatten()
        )
        
        self.hook = ReshapeBodyHook(self.conv)
        hidden = 96
        self.lstm = nn.LSTM(input_size=d_model*2, hidden_size=hidden, num_layers=2, bidirectional=True)
        self.head = nn.Sequential(
#             nn.Linear(hidden*2,hidden*4),
#             nn.ReLU(),
            nn.Linear(hidden*2,6)
        )
        
    def forward(self, x):
        x = self.conv(x)
        feat, _ = self.lstm(x.transpose(0,1))
        return self.head(feat.transpose(0,1))

In [6]:
m = SeqHead()

In [7]:
config=dict(custom_head=m)
learn = get_learner(dls, xresnet18, get_loss(), config=config)

In [8]:
hook = ReshapeBodyHook(learn.model[0])
learn.add_cb(DePadLoss())

In [9]:
learn.load(f'runs/baseline_stg1_xresnet18-3', strict=False)

In [10]:
# learn.summary()

## Training

In [11]:
name = 'train3d_baseline_lstm'

In [12]:
# learn.lr_find()

In [13]:
learn.freeze()
learn.fit_one_cycle(1, 1e-2)
learn.unfreeze()

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.083220,0.089276,0.971613,0.946464,09:35


In [15]:
learn.save(f'runs/{name}-freeze')

## 384 - no_freeze

In [16]:
learn.load(f'runs/{name}-freeze')

In [17]:
learn.dls = get_3d_dls_aug(Meta.df_comb, path=path_jpg, sz=384, bs=4, grps=Meta.grps_stg1)

In [ ]:
do_fit(learn, 4, 1e-4)
learn.save(f'runs/{name}-2-no256')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.060172,0.075798,0.974679,0.954001,1:07:20


## Test

In [9]:
learn.load(f'runs/{name}-2-no256', strict=True)
sub_fn = f'subm/{name}'

In [14]:
dls = get_3d_dls_aug(Meta.df_tst, sz=384, path=path_tst_jpg, bs=8, test=True)

In [15]:
learn = get_learner(dls, learn.model)
hook.deregister()
hook = ReshapeBodyHook(learn.model[0])
learn.add_cb(DePadLoss())

In [ ]:
preds,targs = learn.get_preds()

In [17]:
preds.shape, preds.min(), preds.max()

(torch.Size([121232, 6]), tensor(9.1849e-08), tensor(0.4604))

In [15]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [47]:
pred_csv

,ID,Label
0,ID_714683b15_any,0.0030043430
1,ID_714683b15_epidural,0.0003814712
2,ID_714683b15_intraparenchymal,0.0004373344
3,ID_714683b15_intraventricular,0.0003829637
4,ID_714683b15_subarachnoid,0.0036642202
...,...,...
727387,ID_0abae0d75_epidural,0.0000001841
727388,ID_0abae0d75_intraparenchymal,0.0000002477
727389,ID_0abae0d75_intraventricular,0.0000002919
727390,ID_0abae0d75_subarachnoid,0.0000002785


In [16]:
api.competition_submit(f'{sub_fn}.csv', name, 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.57MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [23]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14285598,
 'totalBytes': 27277209,
 'date': '2020-01-25T03:21:22.64Z',
 'description': 'train3d_baseline_pretrained',
 'errorDescription': None,
 'fileName': 'train3d_baseline_pretrained.csv',
 'publicScore': '2.11174',
 'privateScore': '0.38429',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14285598/14285598.raw'}

In [ ]:
## Test

learn.load(f'runs/{name}-2-no256', strict=True)
sub_fn = f'subm/{name}'

dls = get_3d_dls_aug(Meta.df_tst, sz=384, path=path_tst_jpg, bs=8, test=True)

learn = get_learner(dls, learn.model)
hook.deregister()
hook = ReshapeBodyHook(learn.model[0])
learn.add_cb(DePadLoss())

preds,targs = learn.get_preds()

preds.shape, preds.min(), preds.max()

pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

pred_csv

api.competition_submit(f'{sub_fn}.csv', name, 'rsna-intracranial-hemorrhage-detection')

api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]